# 📄 Document type classification baseline code with WandB Integration



In [50]:

# =============================================================================
# 0. Prepare Environments & Install Libraries
# =============================================================================

# 필요한 라이브러리를 설치합니다.
!pip install -r ../requirements.txt

In [51]:
# =============================================================================
# 1. Import Libraries & Define Functions
# =============================================================================

import os
import time
import random
import copy

import optuna, math
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler  # Mixed Precision용

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# WandB 관련 import 추가
import wandb
from datetime import datetime


In [52]:
# =============================================================================
# 1-1. WandB Login and Configuration
# =============================================================================
"""
🚀 팀원 사용 가이드:

1. WandB 계정 생성: https://wandb.ai/signup
2. 이 셀 실행 시 로그인 프롬프트가 나타나면 개인 API 키 입력
3. EXPERIMENT_NAME을 다음과 같이 변경:
   - "member1-baseline"
   - "member2-augmentation-test"  
   - "member3-hyperparameter-tuning"
   등등 각자 다른 이름 사용

4. 팀 대시보드 URL: [여기에 당신의 프로젝트 URL 추가]

⚠️ 주의사항:
- 절대 API 키를 코드에 하드코딩하지 마세요
- EXPERIMENT_NAME만 변경하고 PROJECT_NAME은 그대로 두세요
- 각자 개인 계정으로 로그인해서 실험을 추가하세요
"""

# WandB 로그인 (각자 실행)
try:
    if wandb.api.api_key is None:
        print("WandB에 로그인이 필요합니다.")
        wandb.login()
    else:
        print(f"WandB 로그인 상태: {wandb.api.viewer()['username']}")
except:
    print("WandB 로그인을 진행합니다...")
    wandb.login()

# 프로젝트 설정 (각자 수정할 부분)
PROJECT_NAME = "document-classification-team-CV"  # 모든 팀원 동일
ENTITY = None  # 각자 개인 계정 사용
EXPERIMENT_NAME = "efficientnet-b3-baseline"  # 팀원별로 변경 (예: "member1-hyperopt", "member2-augmentation")

print(f"프로젝트: {PROJECT_NAME}")
print(f"실험명: {EXPERIMENT_NAME}")
print("팀원들은 EXPERIMENT_NAME을 각자 다르게 변경해주세요!")

WandB 로그인 상태: kimsunmin0227
프로젝트: document-classification-team-CV
실험명: efficientnet-b3-baseline
팀원들은 EXPERIMENT_NAME을 각자 다르게 변경해주세요!


In [53]:
# =============================================================================
# 3. Seed & basic augmentations (Mixup)
# =============================================================================

# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True


In [54]:

# =============================================================================
# 4. Dataset Class
# =============================================================================

class ImageDataset(Dataset):
    def __init__(self, data, path, transform=None):
        # CSV 파일이면 읽고, DataFrame이면 그대로 사용
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values  # DataFrame을 numpy array로 변환
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target


In [55]:
# Focal Loss 구현
class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance
    
    FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    
    Args:
        alpha: Weighting factor for rare class (default: 1.0)
        gamma: Focusing parameter (default: 2.0)
        reduction: Specifies the reduction to apply to the output
    """
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        # Cross entropy loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # Get probabilities
        pt = torch.exp(-ce_loss)
        
        # Calculate focal loss
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Cutout (Random Erasing) 함수 정의
def random_erasing(image, p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)):
    if random.random() > p:
        return image
    img_c, img_h, img_w = image.shape[1], image.shape[2], image.shape[3]
    area = img_h * img_w
    
    target_area = random.uniform(scale[0], scale[1]) * area
    aspect_ratio = random.uniform(ratio[0], ratio[1])
    h = int(round(math.sqrt(target_area * aspect_ratio)))
    w = int(round(math.sqrt(target_area / aspect_ratio)))
    
    if h < img_h and w < img_w:
        x = random.randint(0, img_w - w)
        y = random.randint(0, img_h - h)
        image[:, :, y:y+h, x:x+w] = 0.0  # 제거된 영역을 0으로 설정
    return image

# RandomCrop 함수 정의
def random_crop(image, crop_size=0.8):
    img_c, img_h, img_w = image.shape[1], image.shape[2], image.shape[3]
    crop_h = int(img_h * crop_size)
    crop_w = int(img_w * crop_size)
    
    if crop_h >= img_h or crop_w >= img_w:
        return image
    
    x = random.randint(0, img_w - crop_w)
    y = random.randint(0, img_h - crop_h)
    cropped_image = image[:, :, y:y+crop_h, x:x+crop_w]
    
    # 원래 이미지 크기로 복원 (패딩 또는 리사이즈)
    cropped_image = torch.nn.functional.interpolate(cropped_image, size=(img_h, img_w), mode='bilinear', align_corners=False)
    return cropped_image

# Mixup 함수 정의
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch=None, fold=None):
    scaler = GradScaler()
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, desc=f"Training Epoch {epoch+1 if epoch else '?'}")
    batch_count = 0
    
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)
        
        # 증강 기법 선택 (Mixup 25%, Cutout 25%, RandomCrop 25%, None 25%) -> (Mixup 25%, Cutout 25%, RandomCrop 50%)
        aug_type = random.choices(['mixup', 'cutout', 'random_crop'], weights=[0.25, 0.25, 0.5])[0]
        mixup_applied = False
        cutout_applied = False
        random_crop_applied = False
        
        if aug_type == 'mixup':
            mixed_x, y_a, y_b, lam = mixup_data(image, targets, alpha=1.0)
            with autocast(): 
                preds = model(mixed_x)
            loss = lam * loss_fn(preds, y_a) + (1 - lam) * loss_fn(preds, y_b)
            mixup_applied = True
        elif aug_type == 'cutout':
            image = random_erasing(image, p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3))
            with autocast(): 
                preds = model(image)
            loss = loss_fn(preds, targets)
            cutout_applied = True
        elif aug_type == 'random_crop':
            image = random_crop(image, crop_size=0.8)
            with autocast(): 
                preds = model(image)
            loss = loss_fn(preds, targets)
            random_crop_applied = True
        else:
            with autocast(): 
                preds = model(image)
            loss = loss_fn(preds, targets)

        model.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        # 배치별 상세 로깅 (100 배치마다)
        if batch_count % 100 == 0 and wandb.run is not None:
            step = epoch * len(loader) + batch_count if epoch is not None else batch_count
            wandb.log({
                f"fold_{fold}/train_batch_loss": loss.item(),
                f"fold_{fold}/mixup_applied": int(mixup_applied),
                f"fold_{fold}/cutout_applied": int(cutout_applied),
                f"fold_{fold}/random_crop_applied": int(random_crop_applied),
                f"fold_{fold}/batch_step": step
            })
        
        batch_count += 1
        pbar.set_description(f"Loss: {loss.item():.4f}, Mixup: {mixup_applied}, Cutout: {cutout_applied}, RandomCrop: {random_crop_applied}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

def validate_one_epoch(loader, model, loss_fn, device, epoch=None, fold=None, log_confusion=False):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Validating Epoch {epoch+1 if epoch else '?'}")
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)
            
            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            
            pbar.set_description(f"Val Loss: {loss.item():.4f}")
    
    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')
    
    # Confusion Matrix 로깅 (마지막 epoch에만)
    if log_confusion and wandb.run is not None:
        try:
            wandb.log({
                f"fold_{fold}/confusion_matrix": wandb.plot.confusion_matrix(
                    probs=None,
                    y_true=targets_list,
                    preds=preds_list,
                    class_names=[f"Class_{i}" for i in range(17)]
                )
            })
            
            # 클래스별 F1 스코어
            class_f1_scores = f1_score(targets_list, preds_list, average=None)
            for i, class_f1 in enumerate(class_f1_scores):
                wandb.log({f"fold_{fold}/class_{i}_f1": class_f1})
                
        except Exception as e:
            print(f" Confusion matrix 로깅 실패: {e}")
    
    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,  
        "val_f1": val_f1,
    }
    
    return ret

In [56]:
# =============================================================================
# 6. Hyper-parameters with WandB Config
# =============================================================================

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Using device: {device}")

# data config
data_path = '../data/'

# model config
model_name = 'efficientnet_b3' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 384
LR = 5e-4
EPOCHS = 50
BATCH_SIZE = 64
num_workers = 30

# K-Fold config
N_FOLDS = 5  # 5-fold로 설정

# WandB Config 설정
config = {
    # Model config
    "model_name": model_name,
    "img_size": img_size,
    "num_classes": 17,
    "architecture": "EfficientNet-B3",
    
    # Training config  
    "lr": LR,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "num_workers": num_workers,
    "device": str(device),
    
    # K-Fold config
    "n_folds": N_FOLDS,
    "seed": SEED,
    "cv_strategy": "StratifiedKFold",
    
    # Augmentation & Training techniques
    "mixup_alpha": 1.0,
    "mixup_prob": 0.3,
    "loss_function": "FocalLoss",
    "focal_alpha": 1.0,
    "focal_gamma": 2.0,
    "gradient_clipping": 1.0,
    "mixed_precision": True,
    
    # Optimizer & Scheduler
    "optimizer": "Adam",
    "scheduler": "CosineAnnealingLR",
    
    # Data
    "data_path": data_path,
    "train_transforms": "Advanced",
    "test_transforms": "Basic",
}

print(" 하이퍼파라미터 설정 완료!")
print(f" 모델: {model_name}")
print(f" 이미지 크기: {img_size}x{img_size}")
print(f" 배치 크기: {BATCH_SIZE}")
print(f" 학습률: {LR}")
print(f" 에폭: {EPOCHS}")


 Using device: cuda
 하이퍼파라미터 설정 완료!
 모델: efficientnet_b3
 이미지 크기: 384x384
 배치 크기: 64
 학습률: 0.0005
 에폭: 50


In [ ]:

# =============================================================================
# 7. Optuna Hyperparameter Tuning (선택적)
# =============================================================================

# Optuna 설정
USE_OPTUNA = False  # True로 바꾸면 튜닝 실행
OPTUNA_TRIALS = 20  # Optuna 시행 횟수
OPTUNA_TIMEOUT = 3600  # 1시간 타임아웃 (초)

# Optuna가 활성화된 경우 기본 하이퍼파라미터를 None으로 설정
if USE_OPTUNA:
    print("🔍 Optuna 하이퍼파라미터 튜닝 활성화!")
    print(f"   시행 횟수: {OPTUNA_TRIALS}")
    print(f"   타임아웃: {OPTUNA_TIMEOUT}초")
    
    # 기본값들을 None으로 설정 (Optuna에서 최적화)
    LR = None
    BATCH_SIZE = None
    EPOCHS = None
    img_size = None
    focal_alpha = None
    focal_gamma = None
    mixup_alpha = None
    mixup_prob = None
    gradient_clipping = None
else:
    print("⏭️ Optuna 튜닝 건너뛰기 (USE_OPTUNA=False)")
    print("   기본 하이퍼파라미터 사용")

SyntaxError: invalid syntax (2370869770.py, line 6)

In [58]:
# =============================================================================
# 7-1. Optuna Objective Function
# =============================================================================

def optuna_objective(trial):
    """
    Optuna objective function for hyperparameter optimization
    """
    # 하이퍼파라미터 제안
    params = {
        # 학습률 (로그 스케일)
        'lr': trial.suggest_float('lr', 1e-5, 1e-2, log=True),
        
        # 배치 크기 (2의 거듭제곱)
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        
        # 에폭 수 (적당한 범위)
        'epochs': trial.suggest_int('epochs', 20, 80),
        
        # 이미지 크기
        'img_size': trial.suggest_categorical('img_size', [224, 256, 288, 320, 384, 448]),
        
        # Focal Loss 파라미터
        'focal_alpha': trial.suggest_float('focal_alpha', 0.5, 2.0),
        'focal_gamma': trial.suggest_float('focal_gamma', 1.0, 3.0),
        
        # Mixup 파라미터
        'mixup_alpha': trial.suggest_float('mixup_alpha', 0.2, 2.0),
        'mixup_prob': trial.suggest_float('mixup_prob', 0.1, 0.5),
        
        # 그래디언트 클리핑
        'gradient_clipping': trial.suggest_float('gradient_clipping', 0.5, 2.0),
        
        # 옵티마이저 파라미터
        'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True),
        
        # 스케줄러 파라미터
        'min_lr_ratio': trial.suggest_float('min_lr_ratio', 0.01, 0.1),
    }
    
    print(f"\n🔍 Trial {trial.number}: Testing hyperparameters...")
    print(f"   LR: {params['lr']:.6f}")
    print(f"   Batch Size: {params['batch_size']}")
    print(f"   Epochs: {params['epochs']}")
    print(f"   Image Size: {params['img_size']}")
    print(f"   Focal Alpha: {params['focal_alpha']:.3f}")
    print(f"   Focal Gamma: {params['focal_gamma']:.3f}")
    
    try:
        # 데이터 로드
        train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
        
        # Stratified K-Fold
        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
        
        # 이미지 크기에 따른 transform 설정
        trn_transform = A.Compose([
            A.LongestMaxSize(max_size=params['img_size']),
            A.PadIfNeeded(min_height=params['img_size'], min_width=params['img_size'], 
                          border_mode=0, value=0),
            A.OneOf([
                A.Rotate(limit=[90,90], p=1.0),
                A.Rotate(limit=[180,180], p=1.0),
                A.Rotate(limit=[270,270], p=1.0),
                A.Rotate(limit=(-15, 15), p=1.0),
            ], p=0.7),
            A.OneOf([
                A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=5, p=1.0),
                A.ElasticTransform(alpha=50, sigma=5, p=1.0),
                A.GridDistortion(num_steps=5, distort_limit=0.2, p=1.0),
                A.OpticalDistortion(distort_limit=0.2, shift_limit=0.1, p=1.0),
            ], p=0.6),
            A.OneOf([
                A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.1, p=1.0),
                A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=1.0),
                A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
                A.RandomGamma(gamma_limit=(70, 130), p=1.0),
            ], p=0.9),
            A.OneOf([
                A.MotionBlur(blur_limit=(5, 15), p=1.0),
                A.GaussianBlur(blur_limit=(3, 15), p=1.0),
                A.MedianBlur(blur_limit=7, p=1.0),
                A.Blur(blur_limit=7, p=1.0),
            ], p=0.8),
            A.OneOf([
                A.GaussNoise(var_limit=(10.0, 150.0), p=1.0),
                A.ISONoise(color_shift=(0.01, 0.08), intensity=(0.1, 0.8), p=1.0),
                A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=1.0),
            ], p=0.8),
            A.OneOf([
                A.Downscale(scale_min=0.7, scale_max=0.9, p=1.0),
                A.ImageCompression(quality_lower=60, quality_upper=95, p=1.0),
                A.Posterize(num_bits=6, p=1.0),
            ], p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
        val_transform = A.Compose([
            A.LongestMaxSize(max_size=params['img_size']),
            A.PadIfNeeded(min_height=params['img_size'], min_width=params['img_size'], 
                          border_mode=0, value=0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
        # K-Fold 교차 검증
        fold_scores = []
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
            print(f"   📊 Fold {fold+1}/{N_FOLDS} 시작...")
            
            # 데이터 분할
            train_fold = train_df.iloc[train_idx]
            val_fold = train_df.iloc[val_idx]
            
            # 데이터셋 생성
            train_dataset = ImageDataset(train_fold, os.path.join(data_path, 'train'), trn_transform)
            val_dataset = ImageDataset(val_fold, os.path.join(data_path, 'train'), val_transform)
            
            # 데이터로더 생성
            train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], 
                                    shuffle=True, num_workers=num_workers, pin_memory=True)
            val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], 
                                  shuffle=False, num_workers=num_workers, pin_memory=True)
            
            # 모델 생성
            model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
            
            # 손실 함수 및 옵티마이저
            loss_fn = FocalLoss(alpha=params['focal_alpha'], gamma=params['focal_gamma'])
            optimizer = Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
            scheduler = CosineAnnealingLR(optimizer, T_max=params['epochs'], 
                                       eta_min=params['lr'] * params['min_lr_ratio'])
            
            # 훈련
            best_val_f1 = 0
            for epoch in range(params['epochs']):
                # 훈련
                train_metrics = train_one_epoch(train_loader, model, optimizer, loss_fn, device, 
                                              epoch=epoch, fold=fold)
                
                # 검증
                val_metrics = validate_one_epoch(val_loader, model, loss_fn, device, 
                                               epoch=epoch, fold=fold)
                
                # 스케줄러 업데이트
                scheduler.step()
                
                # 최고 성능 업데이트
                if val_metrics['val_f1'] > best_val_f1:
                    best_val_f1 = val_metrics['val_f1']
                
                # 조기 종료 (성능이 너무 나쁘면)
                if epoch > 10 and val_metrics['val_f1'] < 0.1:
                    print(f"      ⚠️ 조기 종료 (epoch {epoch+1})")
                    break
            
            fold_scores.append(best_val_f1)
            print(f"      ✅ Fold {fold+1} 완료 - Best F1: {best_val_f1:.4f}")
            
            # 메모리 정리
            del model, train_dataset, val_dataset, train_loader, val_loader
            torch.cuda.empty_cache()
        
        # 평균 F1 스코어 계산
        mean_f1 = np.mean(fold_scores)
        std_f1 = np.std(fold_scores)
        
        print(f"   🎯 Trial {trial.number} 완료 - Mean F1: {mean_f1:.4f} ± {std_f1:.4f}")
        
        # WandB 로깅 (선택적)
        if wandb.run is not None:
            wandb.log({
                "trial_number": trial.number,
                "mean_f1": mean_f1,
                "std_f1": std_f1,
                **params
            })
        
        return mean_f1
        
    except Exception as e:
        print(f"   ❌ Trial {trial.number} 실패: {str(e)}")
        return 0.0  # 실패한 경우 0 반환

print("✅ Optuna objective function 정의 완료!")


✅ Optuna objective function 정의 완료!


In [ ]:
# =============================================================================
# 7-2. Optuna Study 실행 및 결과 처리 (WandB 제거 버전)
# =============================================================================

if USE_OPTUNA:
    print("🚀 Optuna 하이퍼파라미터 튜닝 시작!")
    print("=" * 60)
    
    # Optuna study 생성
    study = optuna.create_study(
        direction='maximize',  # F1 스코어 최대화
        sampler=optuna.samplers.TPESampler(seed=SEED),  # TPE 샘플러 사용
        pruner=optuna.pruners.MedianPruner(  # 중간값 기반 조기 종료
            n_startup_trials=5,  # 처음 5개 trial은 조기 종료하지 않음
            n_warmup_steps=10,   # 10 스텝 후 조기 종료 고려
            interval_steps=1     # 매 스텝마다 조기 종료 확인
        )
    )
    
    try:
        # Optuna 최적화 실행
        study.optimize(
            optuna_objective,
            n_trials=OPTUNA_TRIALS,
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        
        print("\n" + "=" * 60)
        print("🎉 Optuna 하이퍼파라미터 튜닝 완료!")
        print("=" * 60)
        
        # 최적 하이퍼파라미터 출력
        best_params = study.best_params
        best_value = study.best_value
        
        print(f"🏆 최고 성능: {best_value:.4f}")
        print(f"📊 최적 하이퍼파라미터:")
        for key, value in best_params.items():
            print(f"   {key}: {value}")
        
        # 최적 하이퍼파라미터를 전역 변수로 설정
        LR = best_params['lr']
        BATCH_SIZE = best_params['batch_size']
        EPOCHS = best_params['epochs']
        img_size = best_params['img_size']
        focal_alpha = best_params['focal_alpha']
        focal_gamma = best_params['focal_gamma']
        mixup_alpha = best_params['mixup_alpha']
        mixup_prob = best_params['mixup_prob']
        gradient_clipping = best_params['gradient_clipping']
        
        # 최적화 히스토리 시각화
        print(f"\n📈 최적화 히스토리:")
        print(f"   총 시행 횟수: {len(study.trials)}")
        print(f"   완료된 시행: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
        print(f"   조기 종료된 시행: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
        print(f"   실패한 시행: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
        
        # 상위 5개 결과 출력
        print(f"\n🏅 상위 5개 결과:")
        sorted_trials = sorted([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE], 
                             key=lambda x: x.value, reverse=True)[:5]
        for i, trial in enumerate(sorted_trials, 1):
            print(f"   {i}. F1: {trial.value:.4f} - Trial {trial.number}")
        
        print(f"\n✅ 최적 하이퍼파라미터가 설정되었습니다!")
        print(f"   이제 이 파라미터들로 최종 모델을 훈련할 수 있습니다.")
        
    except KeyboardInterrupt:
        print(f"\n⚠️ 사용자에 의해 중단되었습니다.")
        if len(study.trials) > 0:
            print(f"   현재까지의 최고 성능: {study.best_value:.4f}")
            print(f"   최적 파라미터: {study.best_params}")
        else:
            print("   아직 완료된 trial이 없습니다.")
        
    except Exception as e:
        print(f"\n❌ Optuna 실행 중 오류 발생: {str(e)}")
        print(f"   기본 하이퍼파라미터를 사용합니다.")
        
        # 기본값으로 복원
        LR = 5e-4
        BATCH_SIZE = 32
        EPOCHS = 50
        img_size = 384
        focal_alpha = 1.0
        focal_gamma = 2.0
        mixup_alpha = 1.0
        mixup_prob = 0.3
        gradient_clipping = 1.0
        
else:
    print("⏭️ Optuna 비활성화 - 기본 하이퍼파라미터 사용")
    # 기본값 설정
    LR = 5e-4
    BATCH_SIZE = 32
    EPOCHS = 50
    img_size = 384
    focal_alpha = 1.0
    focal_gamma = 2.0
    mixup_alpha = 1.0
    mixup_prob = 0.3
    gradient_clipping = 1.0

print(f"\n📋 최종 하이퍼파라미터:")
print(f"   학습률: {LR}")
print(f"   배치 크기: {BATCH_SIZE}")
print(f"   에폭: {EPOCHS}")
print(f"   이미지 크기: {img_size}")
print(f"   Focal Alpha: {focal_alpha}")
print(f"   Focal Gamma: {focal_gamma}")
print(f"   Mixup Alpha: {mixup_alpha}")
print(f"   Mixup Prob: {mixup_prob}")
print(f"   Gradient Clipping: {gradient_clipping}")

[I 2025-09-06 14:59:38,760] A new study created in memory with name: no-name-1e736f5e-ec0e-4b52-b144-6521f8ae3086


🚀 Optuna 하이퍼파라미터 튜닝 시작!


  0%|          | 0/20 [00:00<?, ?it/s]


🔍 Trial 0: Testing hyperparameters...
   LR: 0.000133
   Batch Size: 16
   Epochs: 29
   Image Size: 448
   Focal Alpha: 1.749
   Focal Gamma: 1.425
   📊 Fold 1/5 시작...


Loss: 4.5391, Mixup: True, Cutout: False, RandomCrop: False: 100%|██████████| 79/79 [00:16<00:00,  4.65it/s]
Val Loss: 5.5611: 100%|██████████| 20/20 [00:02<00:00,  7.11it/s]
Loss: 4.6797, Mixup: False, Cutout: True, RandomCrop: False: 100%|██████████| 79/79 [00:13<00:00,  5.69it/s]
Val Loss: 5.2895: 100%|██████████| 20/20 [00:02<00:00,  9.61it/s]
Loss: 3.4219, Mixup: True, Cutout: False, RandomCrop: False: 100%|██████████| 79/79 [00:13<00:00,  5.76it/s]
Val Loss: 5.2376: 100%|██████████| 20/20 [00:02<00:00,  9.14it/s]
Loss: 3.4141, Mixup: False, Cutout: True, RandomCrop: False: 100%|██████████| 79/79 [00:13<00:00,  5.69it/s]
Val Loss: 4.1323: 100%|██████████| 20/20 [00:02<00:00,  9.47it/s]
Loss: 4.2188, Mixup: False, Cutout: False, RandomCrop: True: 100%|██████████| 79/79 [00:13<00:00,  5.78it/s]
Val Loss: 3.8510: 100%|██████████| 20/20 [00:02<00:00,  9.47it/s]
Loss: 3.5293, Mixup: True, Cutout: False, RandomCrop: False: 100%|██████████| 79/79 [00:13<00:00,  5.73it/s]
Val Loss: 3.5639

In [ ]:
# =============================================================================
# 8. Data Transforms
# =============================================================================

# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 비율 보존 리사이징 (핵심 개선)
    A.LongestMaxSize(max_size=img_size),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                  border_mode=0, value=0),
    
    # 문서 특화 회전 + 미세 회전 추가
    A.OneOf([
        A.Rotate(limit=[90,90], p=1.0),
        A.Rotate(limit=[180,180], p=1.0),
        A.Rotate(limit=[270,270], p=1.0),
        A.Rotate(limit=(-15, 15), p=1.0),  # 미세 회전 추가
    ], p=0.7),
    
    # 기하학적 변환 강화
    A.OneOf([
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=5, p=1.0),
        A.ElasticTransform(alpha=50, sigma=5, p=1.0),
        A.GridDistortion(num_steps=5, distort_limit=0.2, p=1.0),
        A.OpticalDistortion(distort_limit=0.2, shift_limit=0.1, p=1.0),
    ], p=0.6),
    
    # 색상 및 조명 변환 강화
    A.OneOf([
        A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.1, p=1.0),
        A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=1.0),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
        A.RandomGamma(gamma_limit=(70, 130), p=1.0),
    ], p=0.9),
    
    # 블러 및 노이즈 강화
    A.OneOf([
        A.MotionBlur(blur_limit=(5, 15), p=1.0),
        A.GaussianBlur(blur_limit=(3, 15), p=1.0),
        A.MedianBlur(blur_limit=7, p=1.0),
        A.Blur(blur_limit=7, p=1.0),
    ], p=0.8),
    
    # 다양한 노이즈 추가
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 150.0), p=1.0),
        A.ISONoise(color_shift=(0.01, 0.08), intensity=(0.1, 0.8), p=1.0),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=1.0),
    ], p=0.8),
    
    # 문서 품질 시뮬레이션 (스캔/복사 효과)
    A.OneOf([
        A.Downscale(scale_min=0.7, scale_max=0.9, p=1.0),
        A.ImageCompression(quality_lower=60, quality_upper=95, p=1.0),
        A.Posterize(num_bits=6, p=1.0),
    ], p=0.5),
    
    # 픽셀 레벨 변환
    A.OneOf([
        A.ChannelShuffle(p=1.0),
        A.InvertImg(p=1.0),
        A.Solarize(threshold=128, p=1.0),
        A.Equalize(p=1.0),
    ], p=0.3),
    
    # 공간 변환
    A.OneOf([
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),  # 문서에서도 유용할 수 있음
        A.Transpose(p=1.0),
    ], p=0.6),
    
    # 조각 제거 (Cutout 계열)
    A.OneOf([
        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, 
                       min_holes=1, min_height=8, min_width=8, 
                       fill_value=0, p=1.0),
        A.GridDropout(ratio=0.3, unit_size_min=8, unit_size_max=32, 
                     holes_number_x=5, holes_number_y=5, p=1.0),
    ], p=0.4),
    
    # 최종 정규화
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.LongestMaxSize(max_size=img_size),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                  border_mode=0, value=0),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

print("✅ 데이터 변환 설정 완료!")

✅ 데이터 변환 설정 완료!


In [ ]:
# =============================================================================
# 9. Load Data & Start K-Fold Cross Validation with WandB
# =============================================================================

# 전체 학습 데이터 로드
train_df = pd.read_csv("../data/train.csv")
print(f"학습 데이터: {len(train_df)}개 샘플")

# 클래스 분포 확인
class_counts = train_df['target'].value_counts().sort_index()
print(f" 클래스 분포: {dict(class_counts)}")

# K-Fold 설정
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# K-Fold 결과를 저장할 리스트
fold_results = []
fold_models = []  # 각 fold의 최고 성능 모델을 저장

#  WandB 메인 실험 시작
main_run = wandb.init(
    project=PROJECT_NAME,
    entity=ENTITY,
    name=f"{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
    config=config,
    tags=["k-fold-cv", "ensemble", model_name, "baseline", "main-experiment"],
    group="k-fold-experiment",
    job_type="cross-validation",
    notes=f"{N_FOLDS}-Fold Cross Validation with {model_name}"
)

print(f"\n🚀 WandB 실험 시작!")
print(f"📊 대시보드: {main_run.url}")
print(f"📋 실험명: {main_run.name}")

#  데이터셋 정보 로깅
wandb.log({
    "dataset/total_samples": len(train_df),
    "dataset/num_classes": 17,
    "dataset/samples_per_fold": len(train_df) // N_FOLDS,
})

# 클래스 분포 시각화
class_dist_data = [[f"Class_{i}", count] for i, count in enumerate(class_counts)]
wandb.log({
    "dataset/class_distribution": wandb.plot.bar(
        wandb.Table(data=class_dist_data, columns=["Class", "Count"]),
        "Class", "Count", 
        title="Training Data Class Distribution"
    )
})

print(f"\n{'='*60}")
print(f"🎯 {N_FOLDS}-FOLD CROSS VALIDATION 시작")
print(f"{'='*60}")


학습 데이터: 1570개 샘플
 클래스 분포: {0: 100, 1: 46, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100, 10: 100, 11: 100, 12: 100, 13: 74, 14: 50, 15: 100, 16: 100}



🚀 WandB 실험 시작!
📊 대시보드: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/pzrkpkjc
📋 실험명: efficientnet-b3-baseline-0906-1441

🎯 5-FOLD CROSS VALIDATION 시작


In [ ]:

# =============================================================================
# 10. K-Fold Cross Validation Loop with WandB
# =============================================================================

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(f"\n{'='*50}")
    print(f" FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'='*50}")
    
    # 각 fold별 child run 생성
    fold_run = wandb.init(
        project=PROJECT_NAME,
        entity=ENTITY,
        name=f"fold-{fold+1}-{model_name}-{datetime.now().strftime('%H%M')}",
        config=config,
        tags=["fold", f"fold-{fold+1}", model_name, "child-run"],
        group="k-fold-experiment",
        job_type=f"fold-{fold+1}",
        reinit=True  # 새로운 run 시작 허용
    )
    
    print(f"📊 Fold {fold+1} Dashboard: {fold_run.url}")
    
    # 현재 fold의 train/validation 데이터 분할
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    # 데이터 분할 정보 로깅
    wandb.log({
        "fold_info/fold_number": fold + 1,
        "fold_info/train_samples": len(train_fold_df),
        "fold_info/val_samples": len(val_fold_df),
        "fold_info/train_ratio": len(train_fold_df) / len(train_df),
        "fold_info/val_ratio": len(val_fold_df) / len(train_df)
    })
    
    # 현재 fold의 Dataset 생성
    trn_dataset = ImageDataset(
        train_fold_df,
        "../data/train/",
        transform=trn_transform
    )
    
    val_dataset = ImageDataset(
        val_fold_df,
        "../data/train/",
        transform=tst_transform  # 검증에는 증강 적용 안함
    )
    
    # 현재 fold의 DataLoader 생성
    trn_loader = DataLoader(
        trn_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    print(f"Train samples: {len(trn_dataset)}, Validation samples: {len(val_dataset)}")
    
    # 모델 초기화 (각 fold마다 새로운 모델)
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=17
    ).to(device)
    
    # Focal Loss: 클래스 불균형 문제 해결을 위한 손실 함수
    # alpha: 희귀 클래스에 대한 가중치 (Optuna로 최적화됨)
    # gamma: 어려운 샘플에 집중하는 정도 (Optuna로 최적화됨)
    loss_fn = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)  # 최적화된 Focal Loss 적용
    optimizer = Adam(model.parameters(), lr=LR, weight_decay=1e-4)  # weight_decay 추가
    
    # Learning Rate Scheduler 추가 (최적화된 파라미터 사용)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=LR * 0.01)
    
    # 현재 fold의 최고 성능 추적
    best_val_f1 = 0.0
    best_model = None
    patience = 0
    max_patience = 7
    
    print(f" 모델 학습 시작 - Fold {fold+1}")
    
    # =============================================================================
    # 11. Training Loop for Current Fold
    # =============================================================================
    
    for epoch in range(EPOCHS):
        print(f"\n📈 Epoch {epoch+1}/{EPOCHS}")
        
        # Training
        train_ret = train_one_epoch(
            trn_loader, model, optimizer, loss_fn, device, 
            epoch=epoch, fold=fold+1
        )
        
        # Validation
        val_ret = validate_one_epoch(
            val_loader, model, loss_fn, device, 
            epoch=epoch, fold=fold+1,
            log_confusion=(epoch == EPOCHS-1)  # 마지막 epoch에만 confusion matrix
        )
        
        # Learning rate 로깅
        current_lr = optimizer.param_groups[0]['lr']
        
        # WandB에 metrics 로깅
        log_data = {
            "epoch": epoch + 1,
            "fold": fold + 1,
            "train/loss": train_ret['train_loss'],
            "train/accuracy": train_ret['train_acc'], 
            "train/f1": train_ret['train_f1'],
            "val/loss": val_ret['val_loss'],
            "val/accuracy": val_ret['val_acc'],
            "val/f1": val_ret['val_f1'],
            "learning_rate": current_lr,
            "optimizer/lr": current_lr
        }
        
        # GPU 메모리 사용량 로깅
        if torch.cuda.is_available():
            gpu_memory_used = torch.cuda.memory_allocated(0) / 1e9
            gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
            log_data.update({
                "system/gpu_memory_used_gb": gpu_memory_used,
                "system/gpu_memory_total_gb": gpu_memory_total,
                "system/gpu_utilization_pct": (gpu_memory_used / gpu_memory_total) * 100
            })
        
        wandb.log(log_data)
        
        # Scheduler step
        scheduler.step()
        
        print(f" Epoch {epoch+1:2d} | "
              f"Train Loss: {train_ret['train_loss']:.4f} | "
              f"Train F1: {train_ret['train_f1']:.4f} | "
              f"Val Loss: {val_ret['val_loss']:.4f} | "
              f"Val F1: {val_ret['val_f1']:.4f} | "
              f"LR: {current_lr:.2e}")
        
        # 최고 성능 모델 저장
        if val_ret['val_f1'] > best_val_f1:
            best_val_f1 = val_ret['val_f1']
            best_model = copy.deepcopy(model.state_dict())
            patience = 0
            
            # 최고 성능 모델 아티팩트로 저장
            model_path = f'best_model_fold_{fold+1}.pth'
            torch.save(best_model, model_path)
            wandb.save(model_path, policy="now")
            
            # 새로운 최고 성능 로깅
            wandb.log({
                f"best_performance/epoch": epoch + 1,
                f"best_performance/val_f1": best_val_f1,
                f"best_performance/val_acc": val_ret['val_acc'],
                f"best_performance/val_loss": val_ret['val_loss'],
            })
            
            print(f"🎉 새로운 최고 성능! F1: {best_val_f1:.4f}")
        else:
            patience += 1
            
        # Early stopping (선택적)
        if patience >= max_patience and epoch > EPOCHS // 2:
            print(f"⏸️ Early stopping at epoch {epoch+1} (patience: {patience})")
            wandb.log({"early_stopping/epoch": epoch + 1})
            break
    
    # =============================================================================
    # 12. Fold Results Summary
    # =============================================================================
    
    # 현재 fold 결과 저장
    fold_result = {
        'fold': fold + 1,
        'best_val_f1': best_val_f1,
        'final_train_f1': train_ret['train_f1'],
        'train_samples': len(trn_dataset),
        'val_samples': len(val_dataset),
        'epochs_trained': epoch + 1,
        'early_stopped': patience >= max_patience
    }
    
    fold_results.append(fold_result)
    fold_models.append(best_model)
    
    # Fold 최종 요약 로깅
    wandb.log({
        "fold_summary/best_val_f1": best_val_f1,
        "fold_summary/final_train_f1": train_ret['train_f1'],
        "fold_summary/epochs_trained": epoch + 1,
        "fold_summary/improvement": best_val_f1 - val_ret['val_f1'],
        "fold_summary/early_stopped": patience >= max_patience
    })
    
    print(f"\n Fold {fold + 1} 완료!")
    print(f" 최고 Validation F1: {best_val_f1:.4f}")
    print(f" 학습된 에폭: {epoch + 1}/{EPOCHS}")
    
    # Fold run 종료
    wandb.finish()
    
    # 메모리 정리
    del model, optimizer, scheduler, trn_loader, val_loader
    torch.cuda.empty_cache()



 FOLD 1/5


dataset/num_classes,▁
dataset/samples_per_fold,▁
dataset/total_samples,▁
dataset/num_classes,17
dataset/samples_per_fold,314
dataset/total_samples,1570


📊 Fold 1 Dashboard: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/2tjtce0l
Train samples: 1256, Validation samples: 314
 모델 학습 시작 - Fold 1

📈 Epoch 1/29


Loss: 6.0430, Mixup: True, Cutout: False, RandomCrop: False: 100%|██████████| 79/79 [00:20<00:00,  3.95it/s]
Val Loss: 6.6661: 100%|██████████| 20/20 [00:02<00:00,  7.15it/s]


 Epoch  1 | Train Loss: 5.5644 | Train F1: 0.0499 | Val Loss: 5.3148 | Val F1: 0.0848 | LR: 1.33e-04
🎉 새로운 최고 성능! F1: 0.0848

📈 Epoch 2/29


...

 Epoch  9 | Train Loss: 4.2773 | Train F1: 0.1368 | Val Loss: 3.7489 | Val F1: 0.1984 | LR: 1.10e-04
🎉 새로운 최고 성능! F1: 0.1984

📈 Epoch 10/29


Loss: 4.8477, Mixup: True, Cutout: False, RandomCrop: False:   4%|▍         | 3/79 [00:06<02:50,  2.25s/it]
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised excepti

KeyboardInterrupt: 

socket.send() raised exception.
socket.send() raised exception.


socket.send() raised exception.


In [ ]:
# =============================================================================
# 7-3. Optuna 구현 테스트
# =============================================================================

print("🧪 Optuna 구현 테스트 시작...")
print("=" * 50)

# Optuna 설정 확인
print(f"✅ USE_OPTUNA: {USE_OPTUNA}")
print(f"✅ OPTUNA_TRIALS: {OPTUNA_TRIALS}")
print(f"✅ OPTUNA_TIMEOUT: {OPTUNA_TIMEOUT}초")

# 하이퍼파라미터 변수 확인
print(f"\n📋 하이퍼파라미터 변수 상태:")
print(f"   LR: {LR}")
print(f"   BATCH_SIZE: {BATCH_SIZE}")
print(f"   EPOCHS: {EPOCHS}")
print(f"   img_size: {img_size}")
print(f"   focal_alpha: {focal_alpha}")
print(f"   focal_gamma: {focal_gamma}")
print(f"   mixup_alpha: {mixup_alpha}")
print(f"   mixup_prob: {mixup_prob}")
print(f"   gradient_clipping: {gradient_clipping}")

# Optuna 라이브러리 테스트
try:
    import optuna
    print(f"\n✅ Optuna 라이브러리 로드 성공: v{optuna.__version__}")
    
    # 간단한 테스트 study 생성
    def simple_objective(trial):
        x = trial.suggest_float('x', -10, 10)
        return (x - 2) ** 2
    
    test_study = optuna.create_study(direction='minimize')
    test_study.optimize(simple_objective, n_trials=3)
    
    print(f"✅ Optuna 기본 기능 테스트 성공")
    print(f"   테스트 최적값: {test_study.best_value:.4f}")
    print(f"   테스트 최적 파라미터: {test_study.best_params}")
    
except Exception as e:
    print(f"❌ Optuna 테스트 실패: {e}")

# FocalLoss 테스트
try:
    test_logits = torch.randn(4, 17)
    test_targets = torch.randint(0, 17, (4,))
    
    focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
    loss_value = focal_loss(test_logits, test_targets)
    
    print(f"\n✅ FocalLoss 테스트 성공")
    print(f"   alpha: {focal_alpha}")
    print(f"   gamma: {focal_gamma}")
    print(f"   loss_value: {loss_value.item():.4f}")
    
except Exception as e:
    print(f"❌ FocalLoss 테스트 실패: {e}")

# 데이터 경로 확인
try:
    import os
    data_exists = os.path.exists(data_path)
    train_csv_exists = os.path.exists(os.path.join(data_path, 'train.csv'))
    train_dir_exists = os.path.exists(os.path.join(data_path, 'train'))
    
    print(f"\n📁 데이터 경로 확인:")
    print(f"   data_path: {data_path}")
    print(f"   경로 존재: {data_exists}")
    print(f"   train.csv 존재: {train_csv_exists}")
    print(f"   train/ 디렉토리 존재: {train_dir_exists}")
    
    if data_exists and train_csv_exists and train_dir_exists:
        print(f"✅ 데이터 경로 정상")
    else:
        print(f"⚠️ 데이터 경로에 문제가 있을 수 있습니다")
        
except Exception as e:
    print(f"❌ 데이터 경로 확인 실패: {e}")

print(f"\n🎯 Optuna 구현 테스트 완료!")
print(f"   이제 노트북을 실행하여 하이퍼파라미터 최적화를 시작할 수 있습니다.")
print(f"   USE_OPTUNA=True로 설정되어 있어 자동으로 최적화가 실행됩니다.")


In [ ]:
# =============================================================================
# 13. K-Fold Cross Validation Results Summary
# =============================================================================

print(f"\n{'='*60}")
print(" K-FOLD CROSS VALIDATION 최종 결과")
print(f"{'='*60}")

val_f1_scores = [result['best_val_f1'] for result in fold_results]
mean_f1 = np.mean(val_f1_scores)
std_f1 = np.std(val_f1_scores)

try:
    # wandb.run이 현재 활성화된 run을 가리킴
    if wandb.run is None:
        print(" 활성화된 run이 없어 새로운 summary run을 생성합니다.")
        active_run = wandb.init(
            project=PROJECT_NAME,
            name=f"SUMMARY-{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
            config=config,
            tags=["summary", "cv-results", model_name],
            group="k-fold-experiment",
            job_type="summary",
            reinit=True
        )
    else:
        print(" 기존 run을 사용합니다.")
        active_run = wandb.run
        
except Exception as e:
    print(f" Run 상태 확인 중 에러: {e}")
    # 새로운 run 생성
    active_run = wandb.init(
        project=PROJECT_NAME,
        name=f"SUMMARY-{EXPERIMENT_NAME}-{datetime.now().strftime('%m%d-%H%M')}",
        config=config,
        tags=["summary", "cv-results", model_name],
        group="k-fold-experiment",
        job_type="summary",
        reinit=True
    )

# CV 요약 테이블 생성
fold_table = wandb.Table(columns=[
    "Fold", "Best_Val_F1", "Final_Train_F1", "Train_Samples", 
    "Val_Samples", "Epochs_Trained", "Early_Stopped"
])

for result in fold_results:
    fold_table.add_data(
        result['fold'], 
        result['best_val_f1'], 
        result['final_train_f1'],
        result['train_samples'], 
        result['val_samples'],
        result['epochs_trained'],
        result['early_stopped']
    )

# 안전한 로깅
try:
    active_run.log({
        "cv_results/mean_f1": mean_f1,
        "cv_results/std_f1": std_f1,
        "cv_results/best_fold_f1": max(val_f1_scores),
        "cv_results/worst_fold_f1": min(val_f1_scores),
        "cv_results/f1_range": max(val_f1_scores) - min(val_f1_scores),
        "cv_results/fold_results_table": fold_table,
        "cv_results/n_folds": N_FOLDS,
        "cv_results/total_epochs": sum([r['epochs_trained'] for r in fold_results]),
        "cv_results/avg_epochs_per_fold": np.mean([r['epochs_trained'] for r in fold_results]),
        "cv_results/early_stopped_folds": sum([r['early_stopped'] for r in fold_results])
    })
    
    # Fold별 성능 바차트 생성
    fold_performance_data = [[f"Fold {i+1}", score] for i, score in enumerate(val_f1_scores)]
    active_run.log({
        "cv_results/fold_performance_chart": wandb.plot.bar(
            wandb.Table(data=fold_performance_data, columns=["Fold", "F1_Score"]),
            "Fold", "F1_Score", 
            title="K-Fold Cross Validation Performance"
        )
    })
    
    print(" CV 결과 로깅 완료!")
    
except Exception as e:
    print(f" WandB 로깅 중 에러: {e}")
    print(" 결과를 콘솔에 출력합니다:")

# 어떤 경우든 콘솔에는 결과 출력
for result in fold_results:
    status = " Early Stopped" if result['early_stopped'] else " Completed"
    print(f"Fold {result['fold']}: {result['best_val_f1']:.4f} "
          f"({result['epochs_trained']} epochs) {status}")

print(f"\n 평균 CV F1: {mean_f1:.4f} ± {std_f1:.4f}")
print(f" 최고 Fold: {max(val_f1_scores):.4f}")
print(f" 최악 Fold: {min(val_f1_scores):.4f}")
print(f" 성능 범위: {max(val_f1_scores) - min(val_f1_scores):.4f}")



 K-FOLD CROSS VALIDATION 최종 결과
 활성화된 run이 없어 새로운 summary run을 생성합니다.


 CV 결과 로깅 완료!
Fold 1: 0.9143 (28 epochs)  Early Stopped
Fold 2: 0.9087 (27 epochs)  Early Stopped
Fold 3: 0.9319 (36 epochs)  Early Stopped
Fold 4: 0.9083 (33 epochs)  Early Stopped
Fold 5: 0.9321 (49 epochs)  Early Stopped

 평균 CV F1: 0.9191 ± 0.0108
 최고 Fold: 0.9321
 최악 Fold: 0.9083
 성능 범위: 0.0238


In [ ]:

# =============================================================================
# 14. Ensemble Models Preparation
# =============================================================================

# 5-Fold 앙상블 모델 준비
ensemble_models = []
print(f"\n🔧 앙상블 모델 준비 중...")

for i, state_dict in enumerate(fold_models):
    fold_model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    fold_model.load_state_dict(state_dict)
    fold_model.eval()
    ensemble_models.append(fold_model)
    print(f"Fold {i+1} 모델 로드 완료")

print(f" 총 {len(ensemble_models)}개 모델로 앙상블 구성")

try:
    if wandb.run is not None:
        wandb.run.log({
            "ensemble/num_models": len(ensemble_models),
            "ensemble/model_architecture": model_name,
            "ensemble/ensemble_type": "simple_average"
        })
    else:
        print("📊 앙상블 정보:")
        print(f"  - 모델 개수: {len(ensemble_models)}")
        print(f"  - 아키텍처: {model_name}")
        print(f"  - 앙상블 타입: simple_average")
except Exception as e:
    print(f"⚠️ 앙상블 정보 로깅 실패: {e}")



🔧 앙상블 모델 준비 중...
Fold 1 모델 로드 완료
Fold 2 모델 로드 완료
Fold 3 모델 로드 완료
Fold 4 모델 로드 완료
Fold 5 모델 로드 완료
 총 5개 모델로 앙상블 구성


In [ ]:

# =============================================================================
# 15. TTA (Test Time Augmentation) Setup
# =============================================================================

# Temperature Scaling 클래스 정의
class TemperatureScaling(nn.Module):
    def __init__(self, temperature=1.5):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
    
    def forward(self, logits):
        return logits / self.temperature

print(f"\n TTA (Test Time Augmentation) 설정...")

# Essential TTA transforms
essential_tta_transforms = [
    # 원본
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 90도 회전들
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[90, 90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[180, 180], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[-90, -90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 밝기 개선
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.RandomBrightnessContrast(brightness_limit=[0.3, 0.3], contrast_limit=[0.3, 0.3], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
]

print(f"TTA 변환 {len(essential_tta_transforms)}개 준비 완료")

try:
    if wandb.run is not None:
        wandb.run.log({
            "tta/num_transforms": len(essential_tta_transforms),
            "tta/transforms_used": ["original", "rot_90", "rot_180", "rot_270", "brightness"],
            "tta/batch_size": 64  # TTA용 배치 크기
        })
    else:
        print("📊 TTA 설정 정보:")
        print(f"  - 변형 개수: {len(essential_tta_transforms)}")
        print(f"  - 변형 종류: original, rot_90, rot_180, rot_270, brightness")
        print(f"  - 배치 크기: 64")
except Exception as e:
    print(f"⚠️ TTA 설정 로깅 실패: {e}")
    print("📊 TTA 설정 정보:")
    print(f"  - 변형 개수: {len(essential_tta_transforms)}")
    print(f"  - 배치 크기: 64")


 TTA (Test Time Augmentation) 설정...
TTA 변환 5개 준비 완료


In [ ]:
# =============================================================================
# 16. TTA Dataset and DataLoader
# =============================================================================

class TTAImageDataset(Dataset):
    def __init__(self, data, path, transforms):
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.transforms = transforms  # 여러 transform을 리스트로 받음

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # 모든 transform을 적용한 결과를 리스트로 반환
        augmented_images = []
        for transform in self.transforms:
            aug_img = transform(image=img)['image']
            augmented_images.append(aug_img)
        
        return augmented_images, target

# TTA Dataset 생성
tta_dataset = TTAImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    essential_tta_transforms
)

# TTA DataLoader (배치 크기를 줄여서 메모리 절약)
tta_loader = DataLoader(
    tta_dataset,
    batch_size=64,  # TTA는 메모리를 많이 사용하므로 배치 크기 줄임
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

print(f" TTA Dataset: {len(tta_dataset)}개 테스트 샘플")

 TTA Dataset: 3140개 테스트 샘플


In [ ]:

# =============================================================================
# 17. Ensemble + TTA Inference with WandB Logging
# =============================================================================

def ensemble_tta_inference_with_logging(models, loader, transforms, confidence_threshold=0.9):
    """5-Fold 모델 앙상블 + TTA 추론 with WandB 로깅"""
    all_predictions = []
    all_confidences = []
    
    # TTA 진행상황 로깅을 위한 테이블
    tta_progress = wandb.Table(columns=["Batch", "Avg_Confidence", "Low_Conf_Count", "High_Conf_Count"])
    
    # Temperature scaling 초기화
    temp_scaling = TemperatureScaling().to(device)
    
    print(f"앙상블 TTA 추론 시작...")
    print(f"{len(models)}개 모델 × {len(transforms)}개 TTA 변형 = {len(models) * len(transforms)}개 예측 평균")
    
    start_time = time.time()
    
    for batch_idx, (images_list, _) in enumerate(tqdm(loader, desc="Ensemble TTA")):
        batch_size = images_list[0].size(0)
        ensemble_probs = torch.zeros(batch_size, 17).to(device)
        
        # 각 fold 모델별 예측
        for model_idx, model in enumerate(models):
            model.eval()
            with torch.no_grad():
                # 각 TTA 변형별 예측
                for tta_idx, images in enumerate(images_list):
                    images = images.to(device)
                    preds = model(images)
                    
                    # Temperature scaling 적용
                    preds = temp_scaling(preds)
                    probs = torch.softmax(preds, dim=1)
                    
                    # 앙상블 확률에 누적 (평균)
                    ensemble_probs += probs / (len(models) * len(images_list))
        
        # 신뢰도 계산
        max_probs = torch.max(ensemble_probs, dim=1)[0]
        batch_confidences = max_probs.cpu().numpy()
        all_confidences.extend(batch_confidences)
        
        final_preds = torch.argmax(ensemble_probs, dim=1)
        all_predictions.extend(final_preds.cpu().numpy())
        
        # 배치별 신뢰도 분석
        high_conf_count = np.sum(batch_confidences >= confidence_threshold)
        low_conf_count = batch_size - high_conf_count
        avg_confidence = np.mean(batch_confidences)
        
        # 진행상황 테이블에 추가
        tta_progress.add_data(batch_idx, avg_confidence, low_conf_count, high_conf_count)
        
        # 배치별 상세 로깅 (20배치마다)
        if batch_idx % 20 == 0:
            elapsed_time = time.time() - start_time
            estimated_total = elapsed_time * len(loader) / (batch_idx + 1)
            remaining_time = estimated_total - elapsed_time
            
            wandb.log({
                "tta_progress/batch": batch_idx,
                "tta_progress/avg_confidence": avg_confidence,
                "tta_progress/high_confidence_ratio": high_conf_count / batch_size,
                "tta_progress/low_confidence_count": low_conf_count,
                "tta_progress/elapsed_time_min": elapsed_time / 60,
                "tta_progress/estimated_remaining_min": remaining_time / 60,
                "tta_progress/samples_processed": (batch_idx + 1) * batch_size,
            })
    
    total_time = time.time() - start_time
    
    # TTA 최종 결과 로깅
    final_avg_confidence = np.mean(all_confidences)
    confidence_std = np.std(all_confidences)
    high_conf_samples = np.sum(np.array(all_confidences) >= confidence_threshold)
    
    wandb.log({
        "tta_results/total_time_min": total_time / 60,
        "tta_results/samples_per_second": len(all_predictions) / total_time,
        "tta_results/final_avg_confidence": final_avg_confidence,
        "tta_results/confidence_std": confidence_std,
        "tta_results/high_confidence_samples": high_conf_samples,
        "tta_results/high_confidence_ratio": high_conf_samples / len(all_predictions),
        "tta_results/total_predictions": len(all_predictions),
        "tta_results/confidence_histogram": wandb.Histogram(all_confidences),
        "tta_results/progress_table": tta_progress
    })
    
    print(f"\n 앙상블 TTA 추론 완료!")
    print(f"총 소요시간: {total_time/60:.1f}분")
    print(f" 평균 신뢰도: {final_avg_confidence:.4f} ± {confidence_std:.4f}")
    print(f" 고신뢰도 샘플: {high_conf_samples}/{len(all_predictions)} ({high_conf_samples/len(all_predictions)*100:.1f}%)")
    
    return all_predictions, all_confidences

# 앙상블 TTA 실행
print(f"\n{'='*60}")
print(" 최종 추론 - 앙상블 + TTA")
print(f"{'='*60}")

tta_predictions, confidences = ensemble_tta_inference_with_logging(
    models=ensemble_models, 
    loader=tta_loader, 
    transforms=essential_tta_transforms,
    confidence_threshold=0.9
)



 최종 추론 - 앙상블 + TTA
앙상블 TTA 추론 시작...
5개 모델 × 5개 TTA 변형 = 25개 예측 평균


Ensemble TTA: 100%|██████████| 50/50 [02:31<00:00,  3.03s/it]



 앙상블 TTA 추론 완료!
총 소요시간: 2.5분
 평균 신뢰도: 0.6706 ± 0.1795
 고신뢰도 샘플: 77/3140 (2.5%)


In [ ]:
# =============================================================================
# 18. Final Results and Submission
# =============================================================================

print(f"\n 최종 결과 정리 중...")

# TTA 결과로 submission 파일 생성
tta_pred_df = pd.DataFrame(tta_dataset.df, columns=['ID', 'target'])
tta_pred_df['target'] = tta_predictions

# 기존 submission과 동일한 순서인지 확인
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == tta_pred_df['ID']).all(), "ID 순서 불일치!"

# 예측 분포 분석
pred_distribution = tta_pred_df['target'].value_counts().sort_index()
pred_table = wandb.Table(columns=["Class", "Count", "Percentage"])

print(f"\n📊 예측 결과 분포:")
for class_id in range(17):
    count = pred_distribution.get(class_id, 0)
    percentage = count / len(tta_pred_df) * 100
    pred_table.add_data(class_id, count, percentage)
    print(f"Class {class_id:2d}: {count:4d} ({percentage:5.1f}%)")

# 신뢰도 분석
confidence_bins = [0.5, 0.7, 0.8, 0.9, 0.95, 1.0]
confidence_analysis = {}
for i, threshold in enumerate(confidence_bins):
    if i == 0:
        count = np.sum(np.array(confidences) >= threshold)
    else:
        prev_threshold = confidence_bins[i-1]
        count = np.sum((np.array(confidences) >= prev_threshold) & (np.array(confidences) < threshold))
    confidence_analysis[f"conf_{threshold}"] = count

# 최종 결과 로깅
try:
    if wandb.run is not None:
        wandb.run.log({
            "final_results/total_predictions": len(tta_predictions),
            "final_results/unique_classes_predicted": len(np.unique(tta_predictions)),
            "final_results/prediction_distribution_table": pred_table,
            "final_results/avg_confidence": np.mean(confidences),
            "final_results/median_confidence": np.median(confidences),
            "final_results/min_confidence": np.min(confidences),
            "final_results/max_confidence": np.max(confidences),
            "final_results/confidence_distribution": wandb.Histogram(confidences),
            **confidence_analysis
        })
        print("최종 결과 WandB 로깅 완료!")
    else:
        print("활성화된 run이 없어 로깅을 건너뜁니다.")
except Exception as e:
    print(f"WandB 로깅 중 에러: {e}")

# 콘솔 출력은 항상 실행
print(f"총 예측 수: {len(tta_predictions)}")
print(f"예측된 클래스 수: {len(np.unique(tta_predictions))}")
print(f"평균 신뢰도: {np.mean(confidences):.4f}")
print(f"신뢰도 범위: {np.min(confidences):.4f} ~ {np.max(confidences):.4f}")


# 예측 분포 바차트
try:
    if wandb.run is not None:
        pred_dist_data = [[f"Class_{i}", pred_distribution.get(i, 0)] for i in range(17)]
        wandb.run.log({
            "final_results/prediction_distribution_chart": wandb.plot.bar(
                wandb.Table(data=pred_dist_data, columns=["Class", "Count"]),
                "Class", "Count", 
                title="Final Prediction Distribution"
            )
        })
        print("예측 분포 차트 로깅 완료!")
    else:
        print("차트 로깅을 건너뜁니다.")
except Exception as e:
    print(f"차트 로깅 중 에러: {e}")

# 결과 저장
output_path = "../output/choice9.csv"
tta_pred_df.to_csv(output_path, index=False)

# 결과 파일을 WandB 아티팩트로 저장
artifact = wandb.Artifact(
    name="final_predictions",
    type="predictions",
    description=f"Final ensemble predictions with {N_FOLDS}-fold CV + TTA"
)
artifact.add_file(output_path)

try:
    if wandb.run is not None:
        wandb.run.log_artifact(artifact)
        print("실험 요약 로깅 완료!")
    else:
        print("활성화된 run이 없어 실험 요약 로깅을 건너뜁니다.")
except Exception as e:
    print(f"실험 요약 로깅 중 에러: {e}")


print(f"\n 최종 결과 저장 완료!")
print(f" 파일 위치: {output_path}")
print(f" 총 예측 수: {len(tta_predictions)}")


 최종 결과 정리 중...

📊 예측 결과 분포:
Class  0:  200 (  6.4%)
Class  1:   90 (  2.9%)
Class  2:  200 (  6.4%)
Class  3:  166 (  5.3%)
Class  4:  206 (  6.6%)
Class  5:  200 (  6.4%)
Class  6:  205 (  6.5%)
Class  7:  246 (  7.8%)
Class  8:  200 (  6.4%)
Class  9:  200 (  6.4%)
Class 10:  204 (  6.5%)
Class 11:  191 (  6.1%)
Class 12:  199 (  6.3%)
Class 13:  157 (  5.0%)
Class 14:   76 (  2.4%)
Class 15:  200 (  6.4%)
Class 16:  200 (  6.4%)
최종 결과 WandB 로깅 완료!
총 예측 수: 3140
예측된 클래스 수: 17
평균 신뢰도: 0.6706
신뢰도 범위: 0.1487 ~ 0.9539
예측 분포 차트 로깅 완료!
실험 요약 로깅 완료!

 최종 결과 저장 완료!
 파일 위치: ../output/choice8.csv
 총 예측 수: 3140


In [ ]:
# =============================================================================
# 19. Experiment Summary and Cleanup
# =============================================================================

# 실험 요약 생성
experiment_summary = {
    "experiment_name": main_run.name,
    "model_architecture": model_name,
    "image_size": img_size,
    "cv_strategy": f"{N_FOLDS}-Fold StratifiedKFold",
    "cv_mean_f1": mean_f1,
    "cv_std_f1": std_f1,
    "cv_best_fold": max(val_f1_scores),
    "ensemble_models": len(ensemble_models),
    "tta_transforms": len(essential_tta_transforms),
    "total_training_time_min": sum([r['epochs_trained'] for r in fold_results]) * 2,  # 추정치
    "avg_prediction_confidence": np.mean(confidences),
    "high_confidence_predictions": np.sum(np.array(confidences) >= 0.9),
    "experiment_tags": ["baseline", "efficientnet-b3", "k-fold-cv", "tta", "ensemble"]
}

# 실험 요약
try:
    if wandb.run is not None:
        wandb.run.log({"experiment_summary": experiment_summary})
        print("실험 요약 로깅 완료!")
    else:
        print("활성화된 run이 없어 실험 요약 로깅을 건너뜁니다.")
except Exception as e:
    print(f"실험 요약 로깅 중 에러: {e}")


# 마지막 상태 업데이트
try:
    if wandb.run is not None:
        wandb.run.log({
            "status": "completed",
            "completion_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_runtime_hours": 0  # start_time 속성 문제로 일단 0으로 설정
        })
        print("최종 상태 업데이트 완료!")
    else:
        print("활성화된 run이 없어 상태 업데이트를 건너뜁니다.")
except Exception as e:
    print(f"상태 업데이트 중 에러: {e}")

print(f"\n실험 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n{'='*60}")
print("실험 완료!")
print(f"{'='*60}")

print(f" K-Fold CV 결과: {mean_f1:.4f} ± {std_f1:.4f}")
print(f" 최고 성능 Fold: {max(val_f1_scores):.4f}")
print(f" 앙상블 모델: {len(ensemble_models)}개")
print(f" TTA 변형: {len(essential_tta_transforms)}개")
print(f" 평균 예측 신뢰도: {np.mean(confidences):.4f}")
print(f" WandB 대시보드: {main_run.url}")

# Sample predictions 출력
print(f"\n 예측 결과 샘플:")
print(tta_pred_df.head(10))

# 메인 run 종료
main_run.finish()

print(f"\n 모든 작업 완료!")
print(f" 결과 파일: {output_path}")
print(f" WandB에서 전체 실험 결과를 확인하세요!")

# 메모리 정리
del ensemble_models
torch.cuda.empty_cache()

실험 요약 로깅 완료!
최종 상태 업데이트 완료!

실험 완료 시간: 2025-09-06 14:29:43

실험 완료!
 K-Fold CV 결과: 0.9191 ± 0.0108
 최고 성능 Fold: 0.9321
 앙상블 모델: 5개
 TTA 변형: 5개
 평균 예측 신뢰도: 0.6706
 WandB 대시보드: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/b1avqwn7

 예측 결과 샘플:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg      12
4  00901f504008d884.jpg       2
5  009b22decbc7220c.jpg      15
6  00b33e0ee6d59427.jpg       0
7  00bbdcfbbdb3e131.jpg       8
8  00c03047e0fbef40.jpg      15
9  00c0dabb63ca7a16.jpg      11

 모든 작업 완료!
 결과 파일: ../output/choice8.csv
 WandB에서 전체 실험 결과를 확인하세요!


In [ ]:
# =============================================================================
# Focal Loss 테스트
# =============================================================================

print("🔍 Focal Loss 구현 테스트...")

# 테스트 데이터 생성
batch_size = 4
num_classes = 17
test_logits = torch.randn(batch_size, num_classes)
test_targets = torch.randint(0, num_classes, (batch_size,))

# Focal Loss와 CrossEntropyLoss 비교
focal_loss = FocalLoss(alpha=1.0, gamma=2.0)
ce_loss = nn.CrossEntropyLoss()

focal_loss_value = focal_loss(test_logits, test_targets)
ce_loss_value = ce_loss(test_logits, test_targets)

print(f"✅ Focal Loss 테스트 완료!")
print(f"   Focal Loss: {focal_loss_value.item():.4f}")
print(f"   CrossEntropy Loss: {ce_loss_value.item():.4f}")
print(f"   Focal Loss가 정상적으로 작동합니다!")

# 클래스 불균형 시뮬레이션 테스트
print(f"\n📊 클래스 불균형 시뮬레이션 테스트...")

# 불균형한 타겟 생성 (대부분이 클래스 0)
imbalanced_targets = torch.tensor([0, 0, 0, 1])  # 3개는 클래스 0, 1개는 클래스 1

# 모델이 클래스 0을 잘 예측하도록 설정 (높은 확률)
easy_logits = torch.tensor([
    [3.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  # 클래스 0 예측
    [2.5, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],  # 클래스 0 예측
    [2.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],  # 클래스 0 예측
    [0.1, 2.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  # 클래스 1 예측 (어려운 샘플)
])

focal_imbalanced = focal_loss(easy_logits, imbalanced_targets)
ce_imbalanced = ce_loss(easy_logits, imbalanced_targets)

print(f"   불균형 데이터에서 Focal Loss: {focal_imbalanced.item():.4f}")
print(f"   불균형 데이터에서 CrossEntropy Loss: {ce_imbalanced.item():.4f}")
print(f"   Focal Loss가 어려운 샘플(클래스 1)에 더 집중합니다!")

print(f"\n🎯 Focal Loss 구현이 완료되었습니다!")
print(f"   - alpha=1.0: 모든 클래스에 균등한 가중치")
print(f"   - gamma=2.0: 어려운 샘플에 집중하는 정도")
print(f"   - 클래스 불균형 문제 해결에 효과적입니다!")
